# 🗄️ ChromaDB 관리 시스템

이 노트북은 ChromaDB를 사용한 벡터 데이터베이스 관리 기능을 제공합니다.

## 📋 주요 기능
1. **ChromaDB 생성 및 초기화**
2. **문서 추가 (텍스트/PDF)**
3. **문서 삭제**
4. **저장된 문서 조회 및 원본 파일명 확인**
5. **VectorDB 상태 확인**

## ⚠️ 사용 전 주의사항
- `.env` 파일에 `OPENAI_API_KEY`가 설정되어 있어야 합니다
- 필요한 패키지: `chromadb`, `langchain`, `openai`, `pypdf` 등

## 🔧 필수 라이브러리 설치 및 임포트

In [ ]:
# 필요한 패키지 설치 (처음 실행 시에만)
# !pip install chromadb langchain-openai langchain-community langchain-chroma python-dotenv pypdf

In [11]:
import os
import shutil
from pathlib import Path
import pandas as pd
from typing import List, Dict, Any

# Langchain imports
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 환경변수 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 모든 라이브러리가 성공적으로 임포트되었습니다.")

✅ 모든 라이브러리가 성공적으로 임포트되었습니다.


In [12]:
class ChromaDBManager:
    """ChromaDB 관리를 위한 클래스"""
    
    def __init__(self, db_path: str = "./chroma_db", embedding_model: str = "text-embedding-3-large"):
        """
        ChromaDB 매니저 초기화
        
        Args:
            db_path: ChromaDB 저장 경로
            embedding_model: 사용할 OpenAI 임베딩 모델
        """
        self.db_path = db_path
        self.embedding_model = embedding_model
        self.embeddings = OpenAIEmbeddings(model=embedding_model)
        self.db = None
        
        print(f"📁 DB 경로: {self.db_path}")
        print(f"🔤 임베딩 모델: {self.embedding_model}")
    
    def check_db_exists(self) -> bool:
        """DB 존재 여부 확인"""
        return os.path.exists(self.db_path) and os.path.isdir(self.db_path) and len(os.listdir(self.db_path)) > 0
    
    def create_new_db(self, documents: list, force_recreate: bool = False) -> bool:
        """
        새로운 ChromaDB 생성
        
        Args:
            documents: 추가할 문서 리스트
            force_recreate: 기존 DB가 있어도 강제로 재생성할지 여부
            
        Returns:
            bool: 성공 여부
        """
        try:
            # 기존 DB 삭제 (필요시)
            if force_recreate and self.check_db_exists():
                print("🗑️ 기존 DB를 삭제하고 새로 생성합니다...")
                self.delete_db()
            
            # 새 DB 생성
            print(f"🔨 새로운 ChromaDB를 생성하고 {len(documents)}개 문서를 추가합니다...")
            
            self.db = Chroma.from_documents(
                documents=documents,
                embedding=self.embeddings,
                collection_name="rag_collection",
                persist_directory=self.db_path
            )
            
            print(f"✅ ChromaDB가 성공적으로 생성되었습니다! ({len(documents)}개 문서)")
            return True
            
        except Exception as e:
            print(f"❌ DB 생성 실패: {str(e)}")
            return False
    
    def load_existing_db(self) -> bool:
        """
        기존 ChromaDB 로드
        
        Returns:
            bool: 성공 여부
        """
        try:
            if not self.check_db_exists():
                print("❌ 로드할 DB가 존재하지 않습니다.")
                return False
            
            print("📥 기존 ChromaDB를 로드합니다...")
            
            self.db = Chroma(
                persist_directory=self.db_path,
                embedding_function=self.embeddings,
                collection_name="rag_collection"
            )
            
            count = self.get_document_count()
            print(f"✅ ChromaDB가 성공적으로 로드되었습니다! ({count}개 문서)")
            return True
            
        except Exception as e:
            print(f"❌ DB 로드 실패: {str(e)}")
            return False
    
    def add_documents(self, documents: list) -> bool:
        """
        기존 DB에 문서 추가
        
        Args:
            documents: 추가할 문서 리스트
            
        Returns:
            bool: 성공 여부
        """
        try:
            if self.db is None:
                if not self.load_existing_db():
                    print("❌ DB를 로드할 수 없어 문서 추가를 실패했습니다.")
                    return False
            
            old_count = self.get_document_count()
            print(f"📝 기존 DB에 {len(documents)}개 문서를 추가합니다...")
            
            self.db.add_documents(documents)
            
            new_count = self.get_document_count()
            print(f"✅ 문서 추가 완료! ({old_count} → {new_count}개)")
            return True
            
        except Exception as e:
            print(f"❌ 문서 추가 실패: {str(e)}")
            return False
    
    def get_document_count(self) -> int:
        """저장된 문서 개수 반환"""
        try:
            if self.db is None:
                return 0
            return self.db._collection.count()
        except:
            return 0
    
    def get_all_documents(self, limit: int = None) -> List[Dict[str, Any]]:
        """
        저장된 모든 문서 조회
        
        Args:
            limit: 조회할 최대 문서 수
            
        Returns:
            List[Dict]: 문서 정보 리스트
        """
        try:
            if self.db is None:
                print("❌ DB가 로드되지 않았습니다.")
                return []
            
            collection = self.db._collection
            
            if limit:
                results = collection.get(limit=limit, include=['documents', 'metadatas'])
            else:
                results = collection.get(include=['documents', 'metadatas'])
            
            documents_info = []
            for i, (doc_id, document, metadata) in enumerate(zip(results['ids'], results['documents'], results['metadatas'])):
                # 원본 파일명 추출
                source = metadata.get('source', 'Unknown') if metadata else 'Unknown'
                filename = os.path.basename(source) if source != 'Unknown' else 'Unknown'
                
                doc_info = {
                    'index': i + 1,
                    'id': doc_id,
                    'content': document,
                    'source_path': source,
                    'filename': filename,
                    'page': metadata.get('page', 'N/A') if metadata else 'N/A',
                    'char_count': len(document),
                    'preview': document[:100] + "..." if len(document) > 100 else document
                }
                documents_info.append(doc_info)
            
            return documents_info
            
        except Exception as e:
            print(f"❌ 문서 조회 실패: {str(e)}")
            return []
    
    def delete_db(self) -> bool:
        """
        ChromaDB 완전 삭제
        
        Returns:
            bool: 성공 여부
        """
        try:
            # DB 연결 해제
            self.db = None
            
            # 가비지 컬렉션
            import gc
            gc.collect()
            
            # 디렉토리 삭제
            if os.path.exists(self.db_path):
                shutil.rmtree(self.db_path)
                print(f"🗑️ ChromaDB가 완전히 삭제되었습니다: {self.db_path}")
            else:
                print("ℹ️ 삭제할 DB가 존재하지 않습니다.")
            
            return True
            
        except Exception as e:
            print(f"❌ DB 삭제 실패: {str(e)}")
            return False
    
    def get_status(self) -> Dict[str, Any]:
        """DB 상태 정보 반환"""
        exists = self.check_db_exists()
        loaded = self.db is not None
        count = self.get_document_count() if loaded else 0
        
        return {
            'db_exists': exists,
            'db_loaded': loaded,
            'document_count': count,
            'db_path': self.db_path,
            'embedding_model': self.embedding_model
        }
    
    def get_files_in_db(self) -> List[str]:
        """DB에 저장된 파일명 목록 반환"""
        try:
            if self.db is None:
                return []
            
            collection = self.db._collection
            results = collection.get(include=['metadatas'])
            
            files_in_db = set()
            for metadata in results['metadatas']:
                if metadata and 'source' in metadata:
                    source = metadata['source']
                    filename = os.path.basename(source)
                    files_in_db.add(filename)
            
            return list(files_in_db)
            
        except Exception as e:
            print(f"❌ DB 파일 목록 조회 실패: {str(e)}")
            return []

print("✅ ChromaDBManager 클래스가 정의되었습니다.")

✅ ChromaDBManager 클래스가 정의되었습니다.


## 🔧 문서 처리 유틸리티 함수

In [13]:
def load_and_split_document(file_path: str, chunk_size: int = 500, chunk_overlap: int = 100) -> list:
    """
    파일을 로드하고 청크로 분할
    
    Args:
        file_path: 파일 경로
        chunk_size: 청크 크기
        chunk_overlap: 청크 겹침
        
    Returns:
        list: 분할된 문서 리스트
    """
    try:
        file_extension = Path(file_path).suffix.lower()
        
        # 파일 타입에 따른 로더 선택
        if file_extension == '.txt':
            loader = TextLoader(file_path, encoding='utf-8')
        elif file_extension == '.pdf':
            loader = PyPDFLoader(file_path)
        else:
            raise ValueError(f"지원하지 않는 파일 형식: {file_extension}")
        
        # 텍스트 분할기 설정
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        
        # 문서 로드 및 분할
        documents = loader.load_and_split(text_splitter)
        
        print(f"📄 파일 로드 성공: {file_path}")
        print(f"📊 생성된 청크 수: {len(documents)}")
        
        return documents
        
    except Exception as e:
        print(f"❌ 파일 로드 실패: {str(e)}")
        return []

def create_sample_documents() -> list:
    """
    테스트용 샘플 문서 생성
    
    Returns:
        list: 샘플 문서 리스트
    """
    from langchain.schema import Document
    
    sample_docs = [
        Document(
            page_content="인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 추론, 문제 해결 등을 수행하는 기술입니다. 머신러닝과 딥러닝이 AI의 핵심 기술로 사용됩니다.",
            metadata={"source": "sample_ai.txt", "page": 1}
        ),
        Document(
            page_content="머신러닝은 데이터로부터 패턴을 학습하여 예측이나 분류를 수행하는 AI의 한 분야입니다. 지도학습, 비지도학습, 강화학습으로 구분됩니다.",
            metadata={"source": "sample_ml.txt", "page": 1}
        ),
        Document(
            page_content="자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하고 처리할 수 있도록 하는 AI 기술입니다. 텍스트 분석, 감정 분석, 번역 등에 활용됩니다.",
            metadata={"source": "sample_nlp.txt", "page": 1}
        )
    ]
    
    print(f"📝 {len(sample_docs)}개의 샘플 문서가 생성되었습니다.")
    return sample_docs

print("✅ 문서 처리 유틸리티 함수가 정의되었습니다.")

✅ 문서 처리 유틸리티 함수가 정의되었습니다.


## 🚀 ChromaDB 초기화 및 기본 설정

In [15]:
# ChromaDB 매니저 인스턴스 생성
db_manager = ChromaDBManager(
    db_path="./chroma_db_test",  # 테스트용 별도 경로
    embedding_model="text-embedding-3-large"
)

# 현재 상태 확인
status = db_manager.get_status()
print("\n📊 현재 ChromaDB 상태:")
print(f"   DB 존재: {'✅' if status['db_exists'] else '❌'}")
print(f"   DB 로드됨: {'✅' if status['db_loaded'] else '❌'}")
print(f"   문서 수: {status['document_count']}개")
print(f"   DB 경로: {status['db_path']}")
print(f"   임베딩 모델: {status['embedding_model']}")

📁 DB 경로: ./chroma_db_test
🔤 임베딩 모델: text-embedding-3-large

📊 현재 ChromaDB 상태:
   DB 존재: ✅
   DB 로드됨: ❌
   문서 수: 0개
   DB 경로: ./chroma_db_test
   임베딩 모델: text-embedding-3-large


## 📝 1. 샘플 문서로 새 DB 생성하기

가장 기본적인 사용법: 샘플 문서로 새로운 ChromaDB를 생성합니다.

In [16]:
# 샘플 문서 생성
sample_documents = create_sample_documents()

# 새 DB 생성 (기존 DB가 있다면 덮어쓰기)
success = db_manager.create_new_db(sample_documents, force_recreate=True)

if success:
    print("\n🎉 샘플 문서로 새 DB가 성공적으로 생성되었습니다!")
    
    # 생성 후 상태 확인
    status = db_manager.get_status()
    print(f"📊 생성된 문서 수: {status['document_count']}개")
else:
    print("❌ DB 생성에 실패했습니다.")

📝 3개의 샘플 문서가 생성되었습니다.
🗑️ 기존 DB를 삭제하고 새로 생성합니다...
❌ DB 삭제 실패: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: './chroma_db_test\\7bcfc8ff-19b7-49f0-afc1-a7e5c2566782\\data_level0.bin'
🔨 새로운 ChromaDB를 생성하고 3개 문서를 추가합니다...
✅ ChromaDB가 성공적으로 생성되었습니다! (3개 문서)

🎉 샘플 문서로 새 DB가 성공적으로 생성되었습니다!
📊 생성된 문서 수: 6개


## 📄 2. 실제 파일에서 문서 추가하기

텍스트 파일이나 PDF 파일을 로드하여 DB에 추가합니다.

## 🚀 Raw Data 동기화 실행

이제 실제로 raw_data 폴더와 ChromaDB를 동기화해보겠습니다.

In [20]:
class RawDataSyncManager:
    """Raw Data 폴더와 ChromaDB 동기화 관리 클래스"""
    
    def __init__(self, raw_data_path: str = "./raw_data"):
        """
        동기화 매니저 초기화
        
        Args:
            raw_data_path: raw_data 폴더 경로
        """
        self.raw_data_path = raw_data_path
        self.supported_extensions = ['.txt', '.pdf']
        
        # raw_data 폴더가 없으면 생성
        if not os.path.exists(self.raw_data_path):
            os.makedirs(self.raw_data_path)
            print(f"📁 raw_data 폴더를 생성했습니다: {self.raw_data_path}")
        else:
            print(f"📁 raw_data 폴더 경로: {self.raw_data_path}")
    
    def scan_raw_data_folder(self) -> List[Dict[str, Any]]:
        """
        raw_data 폴더의 모든 지원 파일 스캔
        
        Returns:
            List[Dict]: 파일 정보 리스트
        """
        files_info = []
        
        try:
            if not os.path.exists(self.raw_data_path):
                print(f"❌ raw_data 폴더가 존재하지 않습니다: {self.raw_data_path}")
                return files_info
            
            for root, dirs, files in os.walk(self.raw_data_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    file_extension = Path(file).suffix.lower()
                    
                    if file_extension in self.supported_extensions:
                        file_stat = os.stat(file_path)
                        
                        file_info = {
                            'filename': file,
                            'full_path': file_path,
                            'relative_path': os.path.relpath(file_path, self.raw_data_path),
                            'extension': file_extension,
                            'size_bytes': file_stat.st_size,
                            'size_mb': round(file_stat.st_size / (1024 * 1024), 2),
                            'modified_time': file_stat.st_mtime,
                            'modified_date': pd.to_datetime(file_stat.st_mtime, unit='s').strftime('%Y-%m-%d %H:%M:%S')
                        }
                        files_info.append(file_info)
            
            return files_info
            
        except Exception as e:
            print(f"❌ raw_data 폴더 스캔 실패: {str(e)}")
            return []
    
    def compare_with_db(self, db_manager: ChromaDBManager) -> Dict[str, List[str]]:
        """
        raw_data 폴더의 파일들과 DB에 저장된 파일들을 비교
        
        Args:
            db_manager: ChromaDBManager 인스턴스
            
        Returns:
            Dict: 동기화 상태 정보
        """
        # raw_data 폴더 파일 목록
        raw_files_info = self.scan_raw_data_folder()
        raw_files = [info['filename'] for info in raw_files_info]
        
        # DB에 저장된 파일 목록
        db_files = db_manager.get_files_in_db()
        
        # 비교 결과
        sync_status = {
            'new_files': [],      # DB에 없는 새 파일들
            'existing_files': [], # DB에 이미 있는 파일들
            'orphaned_files': [], # raw_data에는 없지만 DB에 있는 파일들
            'all_raw_files': raw_files,
            'all_db_files': db_files
        }
        
        # 새 파일과 기존 파일 분류
        for filename in raw_files:
            if filename in db_files:
                sync_status['existing_files'].append(filename)
            else:
                sync_status['new_files'].append(filename)
        
        # 고아 파일 찾기 (DB에만 있고 raw_data에는 없는 파일)
        for filename in db_files:
            if filename not in raw_files:
                sync_status['orphaned_files'].append(filename)
        
        return sync_status
    
    def sync_with_db(self, db_manager: ChromaDBManager, chunk_size: int = 500, chunk_overlap: int = 100) -> bool:
        """
        raw_data 폴더의 새 파일들을 DB에 동기화
        
        Args:
            db_manager: ChromaDBManager 인스턴스
            chunk_size: 텍스트 청크 크기
            chunk_overlap: 청크 겹침
            
        Returns:
            bool: 동기화 성공 여부
        """
        try:
            # 동기화 상태 확인
            sync_status = self.compare_with_db(db_manager)
            new_files = sync_status['new_files']
            
            if not new_files:
                print("✅ 동기화할 새 파일이 없습니다. 모든 파일이 이미 DB에 저장되어 있습니다.")
                return True
            
            print(f"📝 {len(new_files)}개의 새 파일을 DB에 추가합니다...")
            
            # DB가 로드되지 않았다면 로드 시도
            if db_manager.db is None:
                if db_manager.check_db_exists():
                    if not db_manager.load_existing_db():
                        print("❌ 기존 DB 로드에 실패했습니다.")
                        return False
                else:
                    print("ℹ️ 기존 DB가 없습니다. 첫 번째 파일로 새 DB를 생성합니다.")
            
            # 각 새 파일 처리
            total_added_docs = 0
            for filename in new_files:
                file_path = os.path.join(self.raw_data_path, filename)
                
                print(f"\n🔄 처리 중: {filename}")
                
                # 파일 로드 및 분할
                documents = load_and_split_document(file_path, chunk_size, chunk_overlap)
                
                if documents:
                    # DB가 없으면 첫 번째 파일로 생성, 있으면 추가
                    if not db_manager.check_db_exists():
                        success = db_manager.create_new_db(documents)
                    else:
                        success = db_manager.add_documents(documents)
                    
                    if success:
                        total_added_docs += len(documents)
                        print(f"   ✅ {filename}: {len(documents)}개 청크 추가됨")
                    else:
                        print(f"   ❌ {filename}: 추가 실패")
                        return False
                else:
                    print(f"   ⚠️ {filename}: 파일 로드 실패")
            
            print(f"\n🎉 동기화 완료! 총 {total_added_docs}개 청크가 추가되었습니다.")
            return True
            
        except Exception as e:
            print(f"❌ 동기화 실패: {str(e)}")
            return False
    
    def print_sync_report(self, db_manager: ChromaDBManager):
        """
        동기화 상태 리포트 출력
        
        Args:
            db_manager: ChromaDBManager 인스턴스
        """
        print("\n📊 Raw Data 동기화 상태 리포트")
        print("=" * 50)
        
        # raw_data 폴더 스캔
        raw_files_info = self.scan_raw_data_folder()
        sync_status = self.compare_with_db(db_manager)
        
        # raw_data 폴더 정보
        print(f"\n📁 Raw Data 폴더: {self.raw_data_path}")
        print(f"   총 파일 수: {len(raw_files_info)}개")
        
        if raw_files_info:
            total_size_mb = sum(info['size_mb'] for info in raw_files_info)
            print(f"   총 파일 크기: {total_size_mb:.2f} MB")
            
            # 파일 목록 테이블
            df_raw = pd.DataFrame(raw_files_info)
            print("\n📋 Raw Data 파일 목록:")
            display_cols = ['filename', 'extension', 'size_mb', 'modified_date']
            print(df_raw[display_cols].to_string(index=False))
        
        # DB 상태
        db_status = db_manager.get_status()
        print(f"\n🗄️ ChromaDB 상태:")
        print(f"   DB 존재: {'✅' if db_status['db_exists'] else '❌'}")
        print(f"   총 문서 수: {db_status['document_count']}개")
        print(f"   저장된 파일 수: {len(sync_status['all_db_files'])}개")
        
        # 동기화 상태
        print(f"\n🔄 동기화 상태:")
        print(f"   새 파일 (추가 필요): {len(sync_status['new_files'])}개")
        print(f"   기존 파일 (동기화됨): {len(sync_status['existing_files'])}개")
        print(f"   고아 파일 (DB에만 존재): {len(sync_status['orphaned_files'])}개")
        
        # 상세 정보
        if sync_status['new_files']:
            print(f"\n📥 추가할 새 파일들:")
            for filename in sync_status['new_files']:
                print(f"   - {filename}")
        
        if sync_status['orphaned_files']:
            print(f"\n👻 고아 파일들 (raw_data에 없음):")
            for filename in sync_status['orphaned_files']:
                print(f"   - {filename}")
        
        # 권장 사항
        print(f"\n💡 권장 사항:")
        if sync_status['new_files']:
            print(f"   🔄 sync_manager.sync_with_db(db_manager)를 실행하여 새 파일들을 추가하세요")
        if sync_status['orphaned_files']:
            print(f"   🧹 고아 파일들을 DB에서 제거하거나 원본 파일을 raw_data에 복원하는 것을 고려하세요")
        if not sync_status['new_files'] and not sync_status['orphaned_files']:
            print(f"   ✅ 모든 파일이 완벽하게 동기화되어 있습니다!")

print("✅ RawDataSyncManager 클래스가 정의되었습니다.")

✅ RawDataSyncManager 클래스가 정의되었습니다.


In [21]:
# 1. 동기화 관리자들 초기화
print("🔧 관리자 초기화")

# ChromaDB 매니저 (실제 사용할 DB 경로)
db_manager = ChromaDBManager(
    db_path="./chroma_db",  # 실제 DB 경로
    embedding_model="text-embedding-3-large"
)

# Raw Data 동기화 매니저
sync_manager = RawDataSyncManager(
    raw_data_path="./raw_data"
)

print("\n✅ 관리자 초기화 완료!")

🔧 관리자 초기화
📁 DB 경로: ./chroma_db
🔤 임베딩 모델: text-embedding-3-large
📁 raw_data 폴더 경로: ./raw_data

✅ 관리자 초기화 완료!


In [22]:
# 2. 현재 동기화 상태 확인
print("📊 현재 동기화 상태 확인")
sync_manager.print_sync_report(db_manager)

📊 현재 동기화 상태 확인

📊 Raw Data 동기화 상태 리포트

📁 Raw Data 폴더: ./raw_data
   총 파일 수: 2개
   총 파일 크기: 9.60 MB

📋 Raw Data 파일 목록:
                       filename extension  size_mb       modified_date
       2025년 개정세법 해설 내지-12교.pdf      .pdf     8.86 2025-07-15 07:53:04
소득세법(법률)(제20615호)(20250701).pdf      .pdf     0.74 2025-07-15 04:28:14

🗄️ ChromaDB 상태:
   DB 존재: ✅
   총 문서 수: 0개
   저장된 파일 수: 0개

🔄 동기화 상태:
   새 파일 (추가 필요): 2개
   기존 파일 (동기화됨): 0개
   고아 파일 (DB에만 존재): 0개

📥 추가할 새 파일들:
   - 2025년 개정세법 해설 내지-12교.pdf
   - 소득세법(법률)(제20615호)(20250701).pdf

💡 권장 사항:
   🔄 sync_manager.sync_with_db(db_manager)를 실행하여 새 파일들을 추가하세요


In [23]:
# 3. 자동 동기화 실행
print("🔄 자동 동기화 시작")

# 청크 설정 (필요에 따라 조정)
CHUNK_SIZE = 1000      # 청크 크기
CHUNK_OVERLAP = 200    # 청크 겹침

# 동기화 실행
success = sync_manager.sync_with_db(
    db_manager=db_manager,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

if success:
    print("\n🎉 동기화가 성공적으로 완료되었습니다!")
else:
    print("\n❌ 동기화 중 문제가 발생했습니다.")

🔄 자동 동기화 시작
📝 2개의 새 파일을 DB에 추가합니다...
📥 기존 ChromaDB를 로드합니다...
✅ ChromaDB가 성공적으로 로드되었습니다! (0개 문서)

🔄 처리 중: 2025년 개정세법 해설 내지-12교.pdf
📄 파일 로드 성공: ./raw_data\2025년 개정세법 해설 내지-12교.pdf
📊 생성된 청크 수: 497
📝 기존 DB에 497개 문서를 추가합니다...
✅ 문서 추가 완료! (0 → 497개)
   ✅ 2025년 개정세법 해설 내지-12교.pdf: 497개 청크 추가됨

🔄 처리 중: 소득세법(법률)(제20615호)(20250701).pdf
📄 파일 로드 성공: ./raw_data\소득세법(법률)(제20615호)(20250701).pdf
📊 생성된 청크 수: 355
📝 기존 DB에 355개 문서를 추가합니다...
✅ 문서 추가 완료! (497 → 852개)
   ✅ 소득세법(법률)(제20615호)(20250701).pdf: 355개 청크 추가됨

🎉 동기화 완료! 총 852개 청크가 추가되었습니다.

🎉 동기화가 성공적으로 완료되었습니다!


In [24]:
# 4. 동기화 후 상태 재확인
print("\n📊 동기화 후 최종 상태")
sync_manager.print_sync_report(db_manager)

# DB 상태 요약
status = db_manager.get_status()
print(f"\n📈 최종 DB 상태 요약:")
print(f"   총 문서 청크 수: {status['document_count']:,}개")
print(f"   DB 경로: {status['db_path']}")
print(f"   임베딩 모델: {status['embedding_model']}")


📊 동기화 후 최종 상태

📊 Raw Data 동기화 상태 리포트

📁 Raw Data 폴더: ./raw_data
   총 파일 수: 2개
   총 파일 크기: 9.60 MB

📋 Raw Data 파일 목록:
                       filename extension  size_mb       modified_date
       2025년 개정세법 해설 내지-12교.pdf      .pdf     8.86 2025-07-15 07:53:04
소득세법(법률)(제20615호)(20250701).pdf      .pdf     0.74 2025-07-15 04:28:14

🗄️ ChromaDB 상태:
   DB 존재: ✅
   총 문서 수: 852개
   저장된 파일 수: 2개

🔄 동기화 상태:
   새 파일 (추가 필요): 0개
   기존 파일 (동기화됨): 2개
   고아 파일 (DB에만 존재): 0개

💡 권장 사항:
   ✅ 모든 파일이 완벽하게 동기화되어 있습니다!

📈 최종 DB 상태 요약:
   총 문서 청크 수: 852개
   DB 경로: ./chroma_db
   임베딩 모델: text-embedding-3-large


In [27]:
# 5. 동기화된 문서들 검색 테스트
print("\n🔍 동기화된 문서 검색 테스트")

if db_manager.db:
    # 검색 테스트 쿼리들
    test_queries = [
        "소득세",
        "개정세법", 
        "세법 개정",
        "2025년"
    ]
    
    for query in test_queries:
        print(f"\n🔍 검색어: '{query}'")
        try:
            results = db_manager.db.similarity_search(query, k=3)
            
            if results:
                for i, result in enumerate(results, 1):
                    filename = os.path.basename(result.metadata.get('source', 'Unknown'))
                    page = result.metadata.get('page', 'N/A')
                    content_preview = result.page_content[:150] + "..." if len(result.page_content) > 150 else result.page_content
                    
                    print(f"   결과 {i}: {filename} (페이지 {page})")
                    print(f"   내용: {content_preview}")
                    print()
        except Exception as e:
            print(f"❌ 검색 중 오류 발생: {str(e)}")
else:
    print("❌ DB가 로드되지 않아 검색을 수행할 수 없습니다.")


🔍 동기화된 문서 검색 테스트

🔍 검색어: '소득세'
   결과 1: 2025년 개정세법 해설 내지-12교.pdf (페이지 34)
   내용: 소득세법(종합소득세 분야) ￭ 27
소득세법(종합소득세 분야)

   결과 2: 소득세법(법률)(제20615호)(20250701).pdf (페이지 0)
   내용: 법제처                                                            1                                                       국가법령정보센터
소득세법
 
소득세법
[시행 2025. ...

   결과 3: 소득세법(법률)(제20615호)(20250701).pdf (페이지 8)
   내용: 법제처                                                            9                                                       국가법령정보센터
소득세법
1. 제14조에 따라 계산한 각...


🔍 검색어: '개정세법'
   결과 1: 2025년 개정세법 해설 내지-12교.pdf (페이지 87)
   내용: 80 ￭ 2025년 개정세법 해설

   결과 2: 2025년 개정세법 해설 내지-12교.pdf (페이지 33)
   내용: 26 ￭ 2025년 개정세법 해설

   결과 3: 2025년 개정세법 해설 내지-12교.pdf (페이지 311)
   내용: 304 ￭ 2025년 개정세법 해설


🔍 검색어: '세법 개정'
   결과 1: 2025년 개정세법 해설 내지-12교.pdf (페이지 121)
   내용: 114 ￭ 2025년 개정세법 해설

   결과 2: 2025년 개정세법 해설 내지-12교.pdf (페이지 323)
   내용: 316 ￭ 2025년 개정세법 해설

   결과 3: 2025년 개정세법 해설 내지-12교.pdf (페이지 385)
   내용: 378 ￭ 2025년 개정세법 해설




## 🗂️ Raw Data 폴더 동기화 기능

이 섹션에서는 `raw_data` 폴더의 파일들과 ChromaDB를 자동으로 동기화하는 기능을 제공합니다.

In [ ]:
# 실제 파일 경로를 지정하세요 (예시)
# file_path = "./sample_document.txt"  # 또는 "./sample_document.pdf"

# 파일이 없는 경우를 위한 샘플 텍스트 파일 생성
sample_file_path = "./test_document.txt"
sample_content = """
이것은 테스트용 문서입니다.

ChromaDB는 벡터 데이터베이스로, 임베딩을 사용하여 문서를 저장하고 검색할 수 있습니다.
이 시스템은 RAG(Retrieval-Augmented Generation) 시스템 구축에 매우 유용합니다.

주요 특징:
1. 빠른 벡터 검색
2. 간단한 API
3. 로컬 저장 지원
4. 메타데이터 관리

이 문서는 테스트 목적으로 작성되었으며, 실제 사용 시에는 더 의미 있는 내용을 포함해야 합니다.
"""

# 샘플 파일 생성
with open(sample_file_path, 'w', encoding='utf-8') as f:
    f.write(sample_content)

print(f"📝 테스트 파일 생성: {sample_file_path}")

# 파일을 로드하고 청크로 분할
documents = load_and_split_document(
    file_path=sample_file_path,
    chunk_size=200,  # 작은 청크 크기로 테스트
    chunk_overlap=50
)

if documents:
    # 기존 DB에 문서 추가
    success = db_manager.add_documents(documents)
    
    if success:
        print("\n🎉 파일에서 문서가 성공적으로 추가되었습니다!")
        
        # 추가 후 상태 확인
        status = db_manager.get_status()
        print(f"📊 총 문서 수: {status['document_count']}개")
    else:
        print("❌ 문서 추가에 실패했습니다.")
else:
    print("❌ 파일 로드에 실패했습니다.")

## 📋 3. 저장된 문서 조회 및 원본 파일명 확인

DB에 저장된 모든 문서를 조회하고 원본 파일명을 확인합니다.

In [3]:
# 모든 문서 조회
documents = db_manager.get_all_documents()

if documents:
    print(f"📚 총 {len(documents)}개의 문서가 저장되어 있습니다.\n")
    
    # 데이터프레임으로 정리하여 표시
    df_data = []
    for doc in documents:
        df_data.append({
            'No.': doc['index'],
            '파일명': doc['filename'],
            '페이지': doc['page'],
            '문자수': doc['char_count'],
            '미리보기': doc['preview']
        })
    
    df = pd.DataFrame(df_data)
    print("📊 저장된 문서 목록:")
    print(df.to_string(index=False))
    
    # 원본 파일별 통계
    print("\n📈 파일별 문서 통계:")
    file_stats = {}
    for doc in documents:
        filename = doc['filename']
        if filename in file_stats:
            file_stats[filename]['count'] += 1
            file_stats[filename]['total_chars'] += doc['char_count']
        else:
            file_stats[filename] = {
                'count': 1,
                'total_chars': doc['char_count']
            }
    
    for filename, stats in file_stats.items():
        print(f"   📄 {filename}: {stats['count']}개 청크, {stats['total_chars']:,}자")
        
else:
    print("📭 저장된 문서가 없습니다.")

NameError: name 'db_manager' is not defined

## 🔍 4. 특정 문서 상세 내용 보기

저장된 문서 중 하나를 선택하여 상세 내용을 확인합니다.

In [ ]:
# 문서가 있는 경우에만 실행
if documents:
    # 첫 번째 문서 선택 (인덱스 변경 가능)
    selected_index = 0  # 원하는 문서 인덱스로 변경
    
    if selected_index < len(documents):
        selected_doc = documents[selected_index]
        
        print(f"📄 문서 {selected_doc['index']} 상세 정보:")
        print(f"   ID: {selected_doc['id']}")
        print(f"   파일명: {selected_doc['filename']}")
        print(f"   원본 경로: {selected_doc['source_path']}")
        print(f"   페이지: {selected_doc['page']}")
        print(f"   문자 수: {selected_doc['char_count']:,}")
        print(f"\n📝 전체 내용:")
        print("-" * 50)
        print(selected_doc['content'])
        print("-" * 50)
    else:
        print(f"❌ 인덱스 {selected_index}는 범위를 벗어났습니다. (0-{len(documents)-1})")
else:
    print("📭 조회할 문서가 없습니다.")

## 📚 결론 및 다음 단계

이 노트북에서는 ChromaDB의 기본 사용법과 Raw Data 폴더 동기화 기능을 구현했습니다:

### ✅ 완료된 기능들:
- **ChromaDB 기본 관리**: 생성, 로드, 문서 추가/삭제
- **Raw Data 폴더 동기화**: 자동 파일 스캔 및 DB 동기화
- **동기화 상태 모니터링**: 상세한 리포트 및 비교 분석
- **검색 기능 테스트**: 동기화된 문서들의 의미 검색
- **문제 해결 가이드**: 일반적인 문제들과 해결 방법

### 🚀 Raw Data 동기화 시스템 특징:
1. **자동 파일 감지**: raw_data 폴더의 모든 .txt, .pdf 파일 자동 스캔
2. **스마트 동기화**: 새 파일만 선별하여 DB에 추가
3. **고아 파일 감지**: DB에만 있고 raw_data에 없는 파일 식별
4. **상세한 리포트**: 파일 크기, 수정 날짜, 동기화 상태 등 포함
5. **배치 처리**: 여러 파일을 한 번에 효율적으로 처리

### 🔄 일반적인 사용 워크플로우:
```python
# 1. 관리자 초기화
db_manager = ChromaDBManager()
sync_manager = RawDataSyncManager()

# 2. 동기화 상태 확인
sync_manager.print_sync_report(db_manager)

# 3. 새 파일들 자동 동기화
sync_manager.sync_with_db(db_manager)

# 4. 검색 테스트
results = db_manager.db.similarity_search(\"검색어\", k=5)
```

### 💡 고급 사용 팁:
- **청크 크기 조정**: 문서 유형에 따라 CHUNK_SIZE 최적화
- **정기 동기화**: cron job이나 스케줄러로 자동 동기화 설정
- **백업 전략**: 중요한 DB는 정기적으로 백업
- **성능 모니터링**: 대용량 파일 처리 시 메모리 사용량 확인
- **버전 관리**: 파일 수정 시 기존 청크 업데이트 전략 수립

### 🛠️ 다음 단계 권장사항:
1. **실제 데이터 테스트**: 본인의 문서들로 대규모 테스트
2. **RAG 시스템 통합**: Streamlit 앱과 연동
3. **성능 최적화**: 더 큰 데이터셋으로 성능 벤치마크
4. **자동화 스크립트**: 배치 처리용 Python 스크립트 작성
5. **모니터링 시스템**: 동기화 로그 및 알림 시스템 구축

### 🚨 주의사항:
- **API 비용**: OpenAI 임베딩 API 사용량 모니터링 필요
- **파일 크기**: 대용량 PDF는 청크 크기 조정 권장
- **동시성**: 여러 프로세스에서 동시 DB 접근 시 주의
- **백업**: 중요한 데이터는 반드시 백업 후 작업

### 📈 성능 개선 아이디어:
- **파일 변경 감지**: 파일 해시를 이용한 실제 변경 감지
- **점진적 업데이트**: 수정된 파일만 재처리
- **병렬 처리**: 멀티프로세싱으로 대용량 파일 처리
- **캐싱**: 자주 사용되는 임베딩 결과 캐시
- **압축**: 저장 공간 절약을 위한 벡터 압축

이제 raw_data 폴더에 새 문서를 추가하면 자동으로 ChromaDB와 동기화할 수 있습니다! 🎉

In [28]:
# 새로운 매니저 인스턴스 생성 (기존 연결 해제 시뮬레이션)
new_db_manager = ChromaDBManager(
    db_path="./chroma_db_test",
    embedding_model="text-embedding-3-large"
)

# 기존 DB 로드
success = new_db_manager.load_existing_db()

if success:
    print("\n🎉 기존 DB가 성공적으로 로드되었습니다!")
    
    # 로드 후 상태 확인
    status = new_db_manager.get_status()
    print(f"📊 로드된 문서 수: {status['document_count']}개")
    
    # 간단한 검색 테스트
    if new_db_manager.db:
        print("\n🔍 검색 테스트: '인공지능'과 관련된 문서 찾기")
        results = new_db_manager.db.similarity_search("인공지능", k=2)
        
        for i, result in enumerate(results, 1):
            print(f"\n결과 {i}:")
            print(f"   내용: {result.page_content[:100]}...")
            print(f"   소스: {result.metadata.get('source', 'Unknown')}")
else:
    print("❌ 기존 DB 로드에 실패했습니다.")

📁 DB 경로: ./chroma_db_test
🔤 임베딩 모델: text-embedding-3-large
📥 기존 ChromaDB를 로드합니다...
✅ ChromaDB가 성공적으로 로드되었습니다! (6개 문서)

🎉 기존 DB가 성공적으로 로드되었습니다!
📊 로드된 문서 수: 6개

🔍 검색 테스트: '인공지능'과 관련된 문서 찾기

결과 1:
   내용: 인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 추론, 문제 해결 등을 수행하는 기술입니다. 머신러닝과 딥러닝이 AI의 핵심 기술로 사용됩니다....
   소스: sample_ai.txt

결과 2:
   내용: 인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 추론, 문제 해결 등을 수행하는 기술입니다. 머신러닝과 딥러닝이 AI의 핵심 기술로 사용됩니다....
   소스: sample_ai.txt


## 🗑️ 6. 문서 삭제 및 DB 관리

특정 문서를 삭제하거나 전체 DB를 삭제하는 방법입니다.

In [29]:
# 현재 상태 확인
print("🔍 삭제 전 DB 상태:")
status = db_manager.get_status()
print(f"   문서 수: {status['document_count']}개")
print(f"   DB 존재: {'✅' if status['db_exists'] else '❌'}")

# 전체 DB 삭제 확인
print("\n⚠️ 주의: 전체 DB를 삭제하시겠습니까?")
print("실제 운영 환경에서는 신중하게 결정하세요.")

# 테스트 환경이므로 바로 삭제 (실제 사용 시에는 확인 절차 추가)
confirm_delete = True  # 실제 사용 시에는 input()으로 사용자 확인

if confirm_delete:
    success = db_manager.delete_db()
    
    if success:
        print("\n🗑️ DB가 성공적으로 삭제되었습니다!")
        
        # 삭제 후 상태 확인
        status = db_manager.get_status()
        print(f"📊 삭제 후 상태:")
        print(f"   DB 존재: {'✅' if status['db_exists'] else '❌'}")
        print(f"   DB 로드됨: {'✅' if status['db_loaded'] else '❌'}")
    else:
        print("❌ DB 삭제에 실패했습니다.")
else:
    print("✋ DB 삭제가 취소되었습니다.")

# 테스트 파일도 정리
if os.path.exists(sample_file_path):
    os.remove(sample_file_path)
    print(f"🧹 테스트 파일 삭제: {sample_file_path}")

🔍 삭제 전 DB 상태:
   문서 수: 852개
   DB 존재: ✅

⚠️ 주의: 전체 DB를 삭제하시겠습니까?
실제 운영 환경에서는 신중하게 결정하세요.
❌ DB 삭제 실패: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: './chroma_db\\a696d458-d0fa-49e6-b0b5-e0934bcdc16a\\data_level0.bin'
❌ DB 삭제에 실패했습니다.


NameError: name 'sample_file_path' is not defined

## 🛠️ 7. 고급 사용법 및 팁

ChromaDB를 더 효과적으로 사용하기 위한 고급 기능들입니다.

In [30]:
# 다양한 청크 크기로 문서 처리 테스트
print("🧪 다양한 청크 크기 테스트")

chunk_sizes = [100, 300, 500]
test_content = "이것은 청크 크기 테스트를 위한 긴 문서입니다. " * 50

for chunk_size in chunk_sizes:
    # 테스트 파일 생성
    test_file = f"./test_chunk_{chunk_size}.txt"
    with open(test_file, 'w', encoding='utf-8') as f:
        f.write(test_content)
    
    # 문서 로드 및 분할
    docs = load_and_split_document(test_file, chunk_size=chunk_size, chunk_overlap=50)
    
    print(f"   청크 크기 {chunk_size}: {len(docs)}개 청크 생성")
    
    # 테스트 파일 삭제
    os.remove(test_file)

print("\n💡 사용 팁:")
print("1. 청크 크기는 문서 유형과 사용 목적에 따라 조정하세요")
print("2. 너무 작은 청크는 문맥 손실, 너무 큰 청크는 검색 정확도 저하를 야기할 수 있습니다")
print("3. chunk_overlap을 사용하여 문맥 연속성을 유지하세요")
print("4. 정기적으로 DB 백업을 수행하세요")
print("5. 대용량 문서 처리 시에는 배치 처리를 고려하세요")

🧪 다양한 청크 크기 테스트
📄 파일 로드 성공: ./test_chunk_100.txt
📊 생성된 청크 수: 25
   청크 크기 100: 25개 청크 생성
📄 파일 로드 성공: ./test_chunk_300.txt
📊 생성된 청크 수: 6
   청크 크기 300: 6개 청크 생성
📄 파일 로드 성공: ./test_chunk_500.txt
📊 생성된 청크 수: 3
   청크 크기 500: 3개 청크 생성

💡 사용 팁:
1. 청크 크기는 문서 유형과 사용 목적에 따라 조정하세요
2. 너무 작은 청크는 문맥 손실, 너무 큰 청크는 검색 정확도 저하를 야기할 수 있습니다
3. chunk_overlap을 사용하여 문맥 연속성을 유지하세요
4. 정기적으로 DB 백업을 수행하세요
5. 대용량 문서 처리 시에는 배치 처리를 고려하세요


## 🚨 8. 문제 해결 및 디버깅

일반적인 문제들과 해결 방법입니다.

In [31]:
print("🔧 문제 해결 가이드")
print("\n1. OpenAI API 키 문제:")
print("   - .env 파일에 OPENAI_API_KEY가 설정되어 있는지 확인")
print("   - API 키가 유효하고 크레딧이 있는지 확인")

print("\n2. 파일 로드 실패:")
print("   - 파일 경로가 정확한지 확인")
print("   - 파일 인코딩 문제 (UTF-8 권장)")
print("   - 파일 권한 문제")

print("\n3. DB 삭제 실패 (Windows):")
print("   - 다른 프로세스에서 DB 사용 중일 수 있음")
print("   - 노트북 커널 재시작 후 다시 시도")
print("   - 폴더를 수동으로 삭제")

print("\n4. 메모리 부족:")
print("   - 큰 문서는 작은 청크로 분할")
print("   - 배치 처리 사용")
print("   - 가비지 컬렉션 활용")

# 환경 확인
print("\n🔍 현재 환경 확인:")
print(f"   Python 버전: {os.sys.version}")
print(f"   현재 작업 디렉토리: {os.getcwd()}")
print(f"   OpenAI API 키 설정: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")

# 필수 패키지 확인
required_packages = ['chromadb', 'langchain', 'openai', 'pandas']
print("\n📦 필수 패키지 확인:")
for package in required_packages:
    try:
        __import__(package)
        print(f"   {package}: ✅")
    except ImportError:
        print(f"   {package}: ❌ (설치 필요)")

🔧 문제 해결 가이드

1. OpenAI API 키 문제:
   - .env 파일에 OPENAI_API_KEY가 설정되어 있는지 확인
   - API 키가 유효하고 크레딧이 있는지 확인

2. 파일 로드 실패:
   - 파일 경로가 정확한지 확인
   - 파일 인코딩 문제 (UTF-8 권장)
   - 파일 권한 문제

3. DB 삭제 실패 (Windows):
   - 다른 프로세스에서 DB 사용 중일 수 있음
   - 노트북 커널 재시작 후 다시 시도
   - 폴더를 수동으로 삭제

4. 메모리 부족:
   - 큰 문서는 작은 청크로 분할
   - 배치 처리 사용
   - 가비지 컬렉션 활용

🔍 현재 환경 확인:
   Python 버전: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
   현재 작업 디렉토리: c:\Users\JS\DEV\cubefi\rag_lab
   OpenAI API 키 설정: ✅

📦 필수 패키지 확인:
   chromadb: ✅
   langchain: ✅
   openai: ✅
   pandas: ✅


## 📚 결론 및 다음 단계

이 노트북에서는 ChromaDB의 기본적인 사용법을 다뤘습니다:

### ✅ 완료된 기능들:
- ChromaDB 생성 및 초기화
- 문서 추가 (샘플 문서 및 파일에서)
- 기존 DB 로드
- 저장된 문서 조회 및 원본 파일명 확인
- DB 삭제
- 문제 해결 가이드

### 🚀 다음 단계 권장사항:
1. **실제 데이터로 테스트**: 본인의 문서들로 테스트해보세요
2. **RAG 시스템 구축**: 이 DB를 사용하여 질의응답 시스템 구축
3. **성능 최적화**: 더 큰 데이터셋으로 성능 테스트
4. **웹 인터페이스 연동**: Streamlit 앱과 연동
5. **백업/복원 시스템**: 프로덕션 환경을 위한 백업 전략 수립

### 💡 추가 개선 아이디어:
- 문서 유사도 검색 기능
- 메타데이터 기반 필터링
- 배치 처리 시스템
- 성능 모니터링
- 자동 백업 스케줄링